## 1. Setup & Import Libraries

In [1]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import warnings
import sys

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Sklearn
from sklearn.model_selection import train_test_split

# Import custom modules
from src.data.loader import load_raw_data, get_data_info
from src.data.cleaner import (
    clean_data, 
    drop_leakage_columns,
    handle_missing_values,
    handle_outliers,
    handle_adr_outliers,
    get_missing_summary,
    save_artifacts,
    encode_categorical,
    scale_numerical
)
from src.features.builder import (
    create_all_features,
    prepare_for_association_rules,
    get_feature_list
)

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✅ Libraries imported successfully!")
print(f"📁 Project root: {project_root}")

✅ Libraries imported successfully!
📁 Project root: c:\Coding\DataMining\Nhom12_BaiTapLon_DataMining


## 2. Load Configuration & Data

In [2]:
# Load configuration
config_path = project_root / 'configs' / 'params.yaml'
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# Extract parameters
SEED = config['seed']
TEST_SIZE = config['split']['test_size']
TARGET = config['target']

print(f"⚙️ Configuration loaded:")
print(f"   - Random Seed: {SEED}")
print(f"   - Test Size: {TEST_SIZE}")
print(f"   - Target Column: {TARGET}")

⚙️ Configuration loaded:
   - Random Seed: 42
   - Test Size: 0.2
   - Target Column: is_canceled


In [3]:
# Load raw data
data_path = project_root / 'data' / 'raw' / 'hotel_bookings.csv'
df_raw = pd.read_csv(data_path)

print(f"📊 Raw Data Loaded:")
print(f"   Shape: {df_raw.shape}")
print(f"   Memory Usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

📊 Raw Data Loaded:
   Shape: (119390, 32)
   Memory Usage: 104.83 MB


## 3. Thống Kê Trước Tiền Xử Lý

Trước khi xử lý, hãy xem tình trạng dữ liệu hiện tại.

In [4]:
# Overview before cleaning
print("="*70)
print("📋 THỐNG KÊ TRƯỚC TIỀN XỬ LÝ")
print("="*70)

print(f"\n📊 Shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")

# Target distribution
target_dist = df_raw[TARGET].value_counts()
target_pct = df_raw[TARGET].value_counts(normalize=True) * 100
print(f"\n🎯 Target Distribution ({TARGET}):")
print(f"   - Not Canceled (0): {target_dist[0]:,} ({target_pct[0]:.2f}%)")
print(f"   - Canceled (1): {target_dist[1]:,} ({target_pct[1]:.2f}%)")
print(f"   → Imbalance Ratio: {target_dist[0]/target_dist[1]:.2f}:1")

# Missing values
missing_total = df_raw.isnull().sum().sum()
print(f"\n❌ Missing Values Total: {missing_total:,}")

📋 THỐNG KÊ TRƯỚC TIỀN XỬ LÝ

📊 Shape: 119,390 rows × 32 columns

🎯 Target Distribution (is_canceled):
   - Not Canceled (0): 75,166 (62.96%)
   - Canceled (1): 44,224 (37.04%)
   → Imbalance Ratio: 1.70:1

❌ Missing Values Total: 129,425


In [5]:
# Detailed missing values
missing_summary = get_missing_summary(df_raw)
if len(missing_summary) > 0:
    print("\n📋 Chi tiết Missing Values:")
    display(missing_summary.style.format({'Missing_Pct': '{:.2f}%'}))
else:
    print("\n✅ Không có missing values!")


📋 Chi tiết Missing Values:


,Column,Missing_Count,Missing_Pct,Dtype
24,company,112593,94.31%,float64
23,agent,16340,13.69%,float64
13,country,488,0.41%,object
10,children,4,0.00%,float64


## 4. Tiền Xử Lý Dữ Liệu

### 4.1. Loại bỏ Data Leakage Columns

⚠️ **QUAN TRỌNG:** Các cột `reservation_status` và `reservation_status_date` chứa thông tin về kết quả đặt phòng (Check-Out/Canceled/No-Show) - đây là thông tin **sau khi** booking đã kết thúc, gây ra **data leakage**.

In [6]:
# Step 1: Drop leakage columns
print("🔒 BƯỚC 1: Loại bỏ Data Leakage Columns")
print("-"*50)

# Check leakage columns before dropping
leakage_cols = ['reservation_status', 'reservation_status_date']
print("\n⚠️ Các cột gây Data Leakage:")
for col in leakage_cols:
    if col in df_raw.columns:
        print(f"   - {col}: {df_raw[col].nunique()} unique values")
        if col == 'reservation_status':
            print(f"     Values: {df_raw[col].value_counts().to_dict()}")

# Drop leakage columns
df_step1 = drop_leakage_columns(df_raw, leakage_cols, verbose=True)
print(f"\n✅ Shape sau khi drop: {df_step1.shape}")

🔒 BƯỚC 1: Loại bỏ Data Leakage Columns
--------------------------------------------------

⚠️ Các cột gây Data Leakage:
   - reservation_status: 3 unique values
     Values: {'Check-Out': 75166, 'Canceled': 43017, 'No-Show': 1207}
   - reservation_status_date: 926 unique values
✓ Dropped 2 leakage column(s): ['reservation_status', 'reservation_status_date']

✅ Shape sau khi drop: (119390, 30)


### 4.2. Xử Lý Missing Values

In [7]:
# Step 2: Handle missing values
print("🔧 BƯỚC 2: Xử lý Missing Values")
print("-"*50)

# Use automatic strategy (defined in cleaner.py)
df_step2 = handle_missing_values(df_step1, strategy='auto', verbose=True)

# Verify no missing values remain
remaining_missing = df_step2.isnull().sum().sum()
print(f"\n✅ Missing values còn lại: {remaining_missing}")

🔧 BƯỚC 2: Xử lý Missing Values
--------------------------------------------------
HANDLING MISSING VALUES
Initial missing values: 129,425
  → Dropped 'company' column (94% missing)
  → Filled 'agent' with 0 (no agent)
  → Filled 'country' with mode: 'PRT'
  → Filled 'children' with 0
------------------------------------------------------------
Final missing values: 0
Rows: 119390 → 119390

✅ Missing values còn lại: 0


### 4.3. Xử Lý Outliers

In [8]:
# Step 3: Handle outliers
print("📈 BƯỚC 3: Xử lý Outliers")
print("-"*50)

# Special handling for ADR (Average Daily Rate)
print("\n🏷️ ADR Statistics Before:")
print(f"   Min: {df_step2['adr'].min():.2f}")
print(f"   Max: {df_step2['adr'].max():.2f}")
print(f"   Mean: {df_step2['adr'].mean():.2f}")
print(f"   Negative values: {(df_step2['adr'] < 0).sum()}")

# Handle ADR outliers
df_step3a = handle_adr_outliers(df_step2, min_adr=0, max_adr=5000, verbose=True)

# Handle other outliers using IQR method with capping
df_step3 = handle_outliers(
    df_step3a,
    method='iqr',
    threshold=1.5,
    strategy='cap',
    verbose=True
)

print(f"\n✅ Shape sau khi xử lý outliers: {df_step3.shape}")

📈 BƯỚC 3: Xử lý Outliers
--------------------------------------------------

🏷️ ADR Statistics Before:
   Min: -6.38
   Max: 5400.00
   Mean: 101.83
   Negative values: 1
ADR Outliers: 1 negative, 1 extreme (>5000)
  → Replaced 1 negative ADR with median: 94.59
  → Capped 1 extreme ADR at 5000
HANDLING OUTLIERS
Method: IQR, Strategy: CAP
  → lead_time: Capped 3005 outliers [-195.00, 373.00]
  → adr: Capped 3793 outliers [-15.77, 211.06]
  → stays_in_weekend_nights: Capped 265 outliers [-3.00, 5.00]
  → stays_in_week_nights: Capped 3354 outliers [-2.00, 6.00]
  → adults: Capped 29710 outliers [2.00, 2.00]
  → children: Capped 8590 outliers [0.00, 0.00]
  → babies: Capped 917 outliers [0.00, 0.00]
  → days_in_waiting_list: Capped 3698 outliers [0.00, 0.00]
------------------------------------------------------------
Total outliers handled: 53,332
Rows: 119390 → 119390

✅ Shape sau khi xử lý outliers: (119390, 29)


## 5. Feature Engineering

Tạo các features mới từ dữ liệu đã làm sạch.

In [9]:
# Apply all feature engineering
df_features = create_all_features(df_step3, config_path=str(config_path), verbose=True)


🔧 FEATURE ENGINEERING PIPELINE

📊 Basic Features:
✓ Created 'total_guests' = adults + children + babies
   Range: [2.0, 2.0]
✓ Created 'total_nights' = weekend_nights + week_nights
   Range: [0, 11]
✓ Created 'is_family' (has children/babies)
   Family bookings: 0.00%

📊 Discretization:
✓ Created 'lead_time_category' with 6 bins
   Bins: [0, 7, 30, 90, 180, 365, 1000]
   Distribution:
     0-7days: 19,746 (16.5%)
     7-30days: 18,960 (15.9%)
     30-90days: 29,553 (24.8%)
     90-180days: 26,439 (22.1%)
     180-365days: 21,544 (18.0%)
     365+days: 3,148 (2.6%)
✓ Created 'country_grouped' (top 10 + Other)
   Top countries: ['PRT', 'GBR', 'FRA', 'ESP', 'DEU', 'ITA', 'IRL', 'BEL', 'BRA', 'NLD']
   'Other' category: 15.16%
✓ Created 'adr_category' (price category)
   Bins: [0, 50, 100, 150, 200, inf]

📊 Temporal Features:
✓ Created seasonal features:
   - arrival_season: ['Summer', 'Fall', 'Winter', 'Spring']
   - is_summer: 31.4%
   - is_peak_season: 22.2%
✓ Created arrival date feat

In [10]:
# List all new features
feature_list = get_feature_list()

print("\n📋 DANH SÁCH FEATURES MỚI:")
print("="*50)
for category, features in feature_list.items():
    print(f"\n🔹 {category.upper()}:")
    for feat in features:
        if feat in df_features.columns:
            print(f"   ✓ {feat}")
        else:
            print(f"   ✗ {feat} (not created)")


📋 DANH SÁCH FEATURES MỚI:

🔹 BASIC:
   ✓ total_guests
   ✓ total_nights
   ✓ is_family

🔹 DISCRETIZED:
   ✓ lead_time_category
   ✓ country_grouped
   ✓ adr_category

🔹 TEMPORAL:
   ✓ arrival_season
   ✓ arrival_month_num
   ✓ is_summer
   ✓ is_peak_season
   ✓ arrival_date
   ✓ arrival_day_of_week
   ✓ is_weekend_arrival

🔹 GUEST_HISTORY:
   ✓ has_canceled_before
   ✓ is_returning_customer
   ✓ repeated_and_canceled_before
   ✓ total_previous_bookings
   ✓ cancellation_ratio

🔹 ROOM:
   ✓ room_type_changed

🔹 BOOKING:
   ✓ has_special_requests
   ✓ has_booking_changes
   ✓ has_agent
   ✓ is_direct_booking
   ✓ deposit_required
   ✓ requires_parking

🔹 REVENUE:
   ✓ total_revenue
   ✓ revenue_per_guest


In [ ]:
# Quick statistics of new features
new_features = [
    'total_guests', 'total_nights', 'is_family',
    'is_summer', 'is_peak_season', 
    'has_canceled_before', 'is_returning_customer',
    'room_type_changed', 'deposit_required'
]

print("\n📊 THỐNG KÊ FEATURES MỚI:")
print("="*60)

for feat in new_features:
    if feat in df_features.columns:
        if df_features[feat].dtype in ['int64', 'float64']:
            if df_features[feat].nunique() <= 2:  # Binary feature
                pct = df_features[feat].mean() * 100
                print(f"{feat:30s}: {pct:6.2f}% = 1")
            else:
                mean = df_features[feat].mean()
                print(f"{feat:30s}: mean = {mean:.2f}")

## 6. Thống Kê Sau Tiền Xử Lý

In [11]:
# Comparison: Before vs After
print("="*70)
print("📊 SO SÁNH TRƯỚC - SAU TIỀN XỬ LÝ")
print("="*70)

comparison = pd.DataFrame({
    'Metric': ['Rows', 'Columns', 'Missing Values', 'Memory (MB)'],
    'Before': [
        f"{df_raw.shape[0]:,}",
        df_raw.shape[1],
        f"{df_raw.isnull().sum().sum():,}",
        f"{df_raw.memory_usage(deep=True).sum() / 1024**2:.2f}"
    ],
    'After': [
        f"{df_features.shape[0]:,}",
        df_features.shape[1],
        f"{df_features.isnull().sum().sum():,}",
        f"{df_features.memory_usage(deep=True).sum() / 1024**2:.2f}"
    ]
})

display(comparison)

📊 SO SÁNH TRƯỚC - SAU TIỀN XỬ LÝ


,Metric,Before,After
0,Rows,"119,390","119,390"
1,Columns,32,56
2,Missing Values,"129,425",0
3,Memory (MB),104.83,117.17


In [12]:
# Columns removed and added
removed_cols = set(df_raw.columns) - set(df_features.columns)
added_cols = set(df_features.columns) - set(df_raw.columns)

print(f"\n🗑️ Columns REMOVED ({len(removed_cols)}):")
for col in sorted(removed_cols):
    print(f"   - {col}")

print(f"\n➕ Columns ADDED ({len(added_cols)}):")
for col in sorted(added_cols):
    print(f"   + {col}")


🗑️ Columns REMOVED (3):
   - company
   - reservation_status
   - reservation_status_date

➕ Columns ADDED (27):
   + adr_category
   + arrival_date
   + arrival_day_of_week
   + arrival_month_num
   + arrival_season
   + cancellation_ratio
   + country_grouped
   + deposit_required
   + has_agent
   + has_booking_changes
   + has_canceled_before
   + has_special_requests
   + is_direct_booking
   + is_family
   + is_peak_season
   + is_returning_customer
   + is_summer
   + is_weekend_arrival
   + lead_time_category
   + repeated_and_canceled_before
   + requires_parking
   + revenue_per_guest
   + room_type_changed
   + total_guests
   + total_nights
   + total_previous_bookings
   + total_revenue


## 7. Train/Test Split

In [13]:
# Separate features and target
X = df_features.drop(columns=[TARGET])
y = df_features[TARGET]

print(f"📊 Features shape: {X.shape}")
print(f"🎯 Target shape: {y.shape}")
print(f"\n🎯 Target distribution:")
print(y.value_counts())

📊 Features shape: (119390, 55)
🎯 Target shape: (119390,)

🎯 Target distribution:
is_canceled
0    75166
1    44224
Name: count, dtype: int64


In [14]:
# Train/Test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=y
)

print(f"📊 TRAIN/TEST SPLIT (Stratified)")
print("="*50)
print(f"\n🔹 Training Set:")
print(f"   X_train: {X_train.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   Canceled ratio: {y_train.mean()*100:.2f}%")

print(f"\n🔹 Test Set:")
print(f"   X_test: {X_test.shape}")
print(f"   y_test: {y_test.shape}")
print(f"   Canceled ratio: {y_test.mean()*100:.2f}%")

📊 TRAIN/TEST SPLIT (Stratified)

🔹 Training Set:
   X_train: (95512, 55)
   y_train: (95512,)
   Canceled ratio: 37.04%

🔹 Test Set:
   X_test: (23878, 55)
   y_test: (23878,)
   Canceled ratio: 37.04%


## 8. Xử Lý Imbalance (SMOTE)

Dataset có tỷ lệ huỷ ~37%, không quá nghiêm trọng nhưng vẫn nên xử lý.

In [15]:
# Try to import SMOTE
try:
    from imblearn.over_sampling import SMOTE
    SMOTE_AVAILABLE = True
    print("✅ SMOTE is available")
except ImportError:
    SMOTE_AVAILABLE = False
    print("⚠️ imbalanced-learn not installed. Run: pip install imbalanced-learn")
    print("   Skipping SMOTE. Will use class_weight instead.")

✅ SMOTE is available


In [16]:
# Select only numerical columns for SMOTE
# SMOTE requires numerical data
numerical_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"📊 Numerical columns: {len(numerical_cols)}")
print(f"📊 Categorical columns: {len(categorical_cols)}")

if categorical_cols:
    print(f"\n⚠️ Categorical columns found: {categorical_cols[:5]}...")
    print("   Need to encode before SMOTE")

📊 Numerical columns: 40
📊 Categorical columns: 14

⚠️ Categorical columns found: ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment']...
   Need to encode before SMOTE


In [17]:
# Encode categorical for SMOTE (if any)
if categorical_cols:
    # One-hot encode categorical columns
    X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
    X_test_encoded = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)
    
    # Align columns (ensure same columns in train and test)
    missing_cols = set(X_train_encoded.columns) - set(X_test_encoded.columns)
    for col in missing_cols:
        X_test_encoded[col] = 0
    
    extra_cols = set(X_test_encoded.columns) - set(X_train_encoded.columns)
    X_test_encoded = X_test_encoded.drop(columns=list(extra_cols))
    
    # Ensure same column order
    X_test_encoded = X_test_encoded[X_train_encoded.columns]
    
    print(f"✅ Encoded shapes:")
    print(f"   X_train_encoded: {X_train_encoded.shape}")
    print(f"   X_test_encoded: {X_test_encoded.shape}")
else:
    X_train_encoded = X_train
    X_test_encoded = X_test

✅ Encoded shapes:
   X_train_encoded: (95512, 286)
   X_test_encoded: (23878, 286)


In [18]:
# Apply SMOTE if available
if SMOTE_AVAILABLE:
    print("🔄 Applying SMOTE...")
    print("-"*50)
    
    # Drop non-numeric columns for SMOTE
    X_train_numeric = X_train_encoded.select_dtypes(include=[np.number])
    
    # Handle any remaining missing values
    X_train_numeric = X_train_numeric.fillna(X_train_numeric.median())
    
    smote = SMOTE(random_state=SEED)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_numeric, y_train)
    
    print(f"\n📊 Before SMOTE:")
    print(f"   X_train: {X_train_numeric.shape}")
    print(f"   Class 0: {(y_train == 0).sum():,}")
    print(f"   Class 1: {(y_train == 1).sum():,}")
    
    print(f"\n📊 After SMOTE:")
    print(f"   X_train_resampled: {X_train_resampled.shape}")
    print(f"   Class 0: {(y_train_resampled == 0).sum():,}")
    print(f"   Class 1: {(y_train_resampled == 1).sum():,}")
else:
    X_train_resampled = X_train_encoded.select_dtypes(include=[np.number])
    y_train_resampled = y_train
    print("⚠️ SMOTE skipped. Using original data.")

🔄 Applying SMOTE...
--------------------------------------------------

📊 Before SMOTE:
   X_train: (95512, 40)
   Class 0: 60,133
   Class 1: 35,379

📊 After SMOTE:
   X_train_resampled: (120266, 40)
   Class 0: 60,133
   Class 1: 60,133


## 9. Chuẩn Bị Data cho Association Rules

In [19]:
# Prepare data for association rules
df_association = prepare_for_association_rules(df_features, verbose=True)

print(f"\n📊 Association Rules Data:")
print(f"   Shape: {df_association.shape}")
print(f"   Sample columns: {list(df_association.columns)[:10]}")

✓ Prepared data for association rules:
   - Input columns: 11
   - Output features: 47
   - Rows: 119390

📊 Association Rules Data:
   Shape: (119390, 47)
   Sample columns: ['is_family', 'is_returning_customer', 'hotel=City Hotel', 'hotel=Resort Hotel', 'arrival_season=Fall', 'arrival_season=Spring', 'arrival_season=Summer', 'arrival_season=Winter', 'meal=BB', 'meal=FB']


## 10. Lưu Dữ Liệu Đã Xử Lý

In [20]:
# Create processed data directory
processed_dir = project_root / 'data' / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Saving processed data to: {processed_dir}")
print("="*50)

📁 Saving processed data to: c:\Coding\DataMining\Nhom12_BaiTapLon_DataMining\data\processed


In [21]:
# Save all datasets

# 1. Full processed data (with features, before encoding)
df_features.to_csv(processed_dir / 'hotel_bookings_processed.csv', index=False)
print(f"✅ Saved: hotel_bookings_processed.csv ({df_features.shape})")

# 2. Train set (original)
train_df = pd.concat([X_train, y_train], axis=1)
train_df.to_csv(processed_dir / 'train.csv', index=False)
print(f"✅ Saved: train.csv ({train_df.shape})")

# 3. Test set (original)
test_df = pd.concat([X_test, y_test], axis=1)
test_df.to_csv(processed_dir / 'test.csv', index=False)
print(f"✅ Saved: test.csv ({test_df.shape})")

# 4. Train set encoded (for modeling)
X_train_encoded.to_csv(processed_dir / 'X_train_encoded.csv', index=False)
y_train.to_csv(processed_dir / 'y_train.csv', index=False)
print(f"✅ Saved: X_train_encoded.csv ({X_train_encoded.shape})")
print(f"✅ Saved: y_train.csv ({y_train.shape})")

# 5. Test set encoded (for modeling)
X_test_encoded.to_csv(processed_dir / 'X_test_encoded.csv', index=False)
y_test.to_csv(processed_dir / 'y_test.csv', index=False)
print(f"✅ Saved: X_test_encoded.csv ({X_test_encoded.shape})")
print(f"✅ Saved: y_test.csv ({y_test.shape})")

# 6. Resampled training data (if SMOTE was applied)
if SMOTE_AVAILABLE:
    X_train_resampled.to_csv(processed_dir / 'X_train_resampled.csv', index=False)
    pd.Series(y_train_resampled, name=TARGET).to_csv(processed_dir / 'y_train_resampled.csv', index=False)
    print(f"✅ Saved: X_train_resampled.csv ({X_train_resampled.shape})")
    print(f"✅ Saved: y_train_resampled.csv ({len(y_train_resampled)},)")

# 7. Association rules data
df_association.to_csv(processed_dir / 'association_rules_data.csv', index=False)
print(f"✅ Saved: association_rules_data.csv ({df_association.shape})")

✅ Saved: hotel_bookings_processed.csv ((119390, 56))
✅ Saved: train.csv ((95512, 56))
✅ Saved: test.csv ((23878, 56))
✅ Saved: X_train_encoded.csv ((95512, 286))
✅ Saved: y_train.csv ((95512,))
✅ Saved: X_test_encoded.csv ((23878, 286))
✅ Saved: y_test.csv ((23878,))
✅ Saved: X_train_resampled.csv ((120266, 40))
✅ Saved: y_train_resampled.csv (120266,)
✅ Saved: association_rules_data.csv ((119390, 47))


## 11. Summary

In [22]:
print("\n" + "="*70)
print("📝 SUMMARY - TIỀN XỬ LÝ & FEATURE ENGINEERING")
print("="*70)

print("\n✅ TIỀN XỬ LÝ HOÀN THÀNH:")
print(f"   1. Loại bỏ Data Leakage columns: {list(removed_cols & set(leakage_cols))}")
print(f"   2. Xử lý Missing values: {missing_total:,} → 0")
print(f"   3. Xử lý Outliers: IQR method với capping")

print(f"\n✅ FEATURE ENGINEERING:")
print(f"   - Columns ban đầu: {df_raw.shape[1]}")
print(f"   - Columns sau xử lý: {df_features.shape[1]}")
print(f"   - Features mới: {len(added_cols)}")

print(f"\n✅ TRAIN/TEST SPLIT:")
print(f"   - Train: {len(y_train):,} samples ({(1-TEST_SIZE)*100:.0f}%)")
print(f"   - Test: {len(y_test):,} samples ({TEST_SIZE*100:.0f}%)")

if SMOTE_AVAILABLE:
    print(f"\n✅ SMOTE RESAMPLING:")
    print(f"   - Before: {len(y_train):,} samples")
    print(f"   - After: {len(y_train_resampled):,} samples")

print(f"\n✅ FILES ĐÃ LƯU:")
for f in processed_dir.glob('*.csv'):
    size_mb = f.stat().st_size / 1024**2
    print(f"   - {f.name}: {size_mb:.2f} MB")

print("\n" + "="*70)
print("🎉 NOTEBOOK HOÀN THÀNH!")
print("   Tiếp theo: 03_mining_or_clustering.ipynb")
print("="*70)


📝 SUMMARY - TIỀN XỬ LÝ & FEATURE ENGINEERING

✅ TIỀN XỬ LÝ HOÀN THÀNH:
   1. Loại bỏ Data Leakage columns: ['reservation_status', 'reservation_status_date']
   2. Xử lý Missing values: 129,425 → 0
   3. Xử lý Outliers: IQR method với capping

✅ FEATURE ENGINEERING:
   - Columns ban đầu: 32
   - Columns sau xử lý: 56
   - Features mới: 27

✅ TRAIN/TEST SPLIT:
   - Train: 95,512 samples (80%)
   - Test: 23,878 samples (20%)

✅ SMOTE RESAMPLING:
   - Before: 95,512 samples
   - After: 120,266 samples

✅ FILES ĐÃ LƯU:
   - association_rules_data.csv: 31.19 MB
   - hotel_bookings_processed.csv: 25.01 MB
   - test.csv: 5.00 MB
   - train.csv: 20.01 MB
   - X_test_encoded.csv: 31.08 MB
   - X_train_encoded.csv: 143.99 MB
   - X_train_resampled.csv: 12.95 MB
   - y_test.csv: 0.07 MB
   - y_train.csv: 0.27 MB
   - y_train_resampled.csv: 0.34 MB

🎉 NOTEBOOK HOÀN THÀNH!
   Tiếp theo: 03_mining_or_clustering.ipynb
